# Compute electric mix for all countries

The main pipeline of `ecodynelec` presented in tutorials [1](https://ecodynelec.readthedocs.io/en/latest/examples/with_python.html#execution) and [2](https://ecodynelec.readthedocs.io/en/latest/examples/with_spreadsheet.html#execution) only compute the electricity mix and impacts for one target country.

However `ecodynelec` can compute the <u>electric mix</u> for all considered countries, ***all at once***. In facts, this information is an intermediate result in the main pipeline. The feature of extracting this intermediate result was added due to its potential usefulness. This tutorial shows how.

In technical words, a function `get_inverted_matrix` allow to retrieve the invert of technology matrix $(I-A)^{-1}$.

In [ ]:
from ecodynelec.pipelines import get_inverted_matrix # Import the function
from ecodynelec.parameter import Parameter # Import the parameter handler object

In [ ]:
# Configuration to reduce the number of time steps
my_config = Parameter().from_excel("./Spreadsheet_example.xlsx")
my_config.start="2017-02-01 12:00"
my_config.end="2017-02-01 14:00" # Only 2 hours
my_config.freq = "H" # Only in hourly time step
my_config.path.generation = "./test_data/generations/" # Generation files
my_config.path.exchanges = "./test_data/exchanges/" # Exchanges files

In [ ]:
# Execute the function
mix = get_inverted_matrix(config=my_config)

The result is **one matrix per time step**, all stored in a `list`. Here is an overview of the first matrix, i.e. corresponding to the first time step.

In [ ]:
display(mix[0])

## Build function to condense and visualize data

In [ ]:
import numpy as np # need numpy functions 
import pandas as pd # need pandas functions

In [ ]:
def group_by_family(matrix):
    families = {'Fossil':['Fossil_Brown_coal/Lignite', 'Fossil_Coal-derived_gas',
                          'Fossil_Gas', 'Fossil_Hard_coal', 'Fossil_Oil', 'Fossil_Oil_shale',
                          'Fossil_Peat','Other_fossil'],
                'Nuclear':['Nuclear'],
                'Hydro':['Hydro_Pumped_Storage','Hydro_Run-of-river_and_poundage',
                         'Hydro_Water_Reservoir', 'Marine'],
                'Other Renwable':['Biomass', 'Geothermal', 'Other_renewable',
                                  'Solar', 'Waste', 'Wind_Offshore', 'Wind_Onshore'],
                'External imports':['Mix']}
    
    per_family = []
    for f in families:
        idx = [k for k in matrix.index if "_".join(k.split("_")[:-1]) in families[f]]
        per_family.append(pd.Series(matrix.loc[idx].sum(), name=f))
    return pd.concat(per_family, axis=1)


def group_per_country(matrix):
    """Function to group datasets per country"""
    countries = np.unique([k.split("_")[-1] for k in matrix.index])
    
    per_country = []
    for c in countries:
        idx = [k for k in matrix.index if k.split("_")[-1]==c]
        per_country.append(pd.Series(matrix.loc[idx].sum(), name=c))
        
    return pd.concat(per_country,axis=1)

## Visualize the origin per production type (columns) in each country (index)

In [ ]:
# Visualize table grouped by type of plant for 1st time step
group_by_family(mix[0].loc["Mix_Other":, :"Mix_IT"])

In [ ]:
# Visualize table grouped by type of plant for 2st time step
group_by_family(mix[1].loc["Mix_Other":, :"Mix_IT"])

## Visualize the origin per country (columns) in each country (index)

In [ ]:
# Visualize table grouped by country for 1st time step
group_per_country(mix[0].loc["Mix_Other":, :"Mix_IT"]) # 1st time step

In [ ]:
# Visualize table grouped by country for 2nd time step
group_per_country(mix[1].loc["Mix_Other":, :"Mix_IT"]) # 2nd time step